In [1]:
# Setup Distributed Client
autostart = True
if autostart:
    pass # just let it use the dask lab extension cluster
else:
    from dask.diagnostics import ProgressBar
    from dask.distributed import Client
    from dask.distributed import LocalCluster
    # client = Client('localhost:8787') # Connect to old dashboard?
    try:
        client = Client('tcp://localhost:8786', timeout='2s')
    except OSError:
        cluster = LocalCluster(scheduler_port=8786)
        client = Client(cluster)
        
client

Client Scheduler: tcp://127.0.0.1:43845 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 24 Memory: 134.93 GB


In [2]:
import xarray as xr
from xarray import open_mfdataset
import netCDF4 as netcdf
import numpy as np
import datetime
import sys
import os

In [3]:
# see https://www.unidata.ucar.edu/software/tds/current/reference/Services.html
def retrieve_data(  data_source =  "https://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USWC/2km/hourly/RTV/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best.ncd", # works,  OPeNDAP DAP2 (ncss->dodsC)
                    #data_source = "https://hfrnet-tds.ucsd.edu/thredds/ncss/HFR/USWC/2km/hourly/RTV/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best.ncd", # doesn't work, NetCDF Subset Service, see see https://hfrnet-tds.ucsd.edu/thredds/ncss/grid/HFR/USWC/2km/hourly/RTV/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best.ncd/dataset.html
                    #data_source = "http://hfrnet-tds.ucsd.edu/thredds/dodsC/HFR/USEGC/6km/hourly/RTV/HFRADAR_US_East_and_Gulf_Coast_6km_Resolution_Hourly_RTV_best.ncd" # 6km allegedly
                    filename_prefix = 'HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best',
                    save_path = "../Data/HF_Radar/2km/",
                    verbose=True,
                    overwrite=False):
    
    xr_data = xr.open_dataset(data_source, decode_times=True,chunks={'time':1000}) # decoding times seems to work, dask chunks for speed
    
    def print_verbose(x,verbose=verbose):
        if verbose: print(x)
        return
#     script_path = os.path.abspath(os.path.dirname(__file__)),# works locally but not in notebook
    script_path = os.path.abspath(os.path.dirname(''))
    save_path = os.path.normpath(os.path.join(script_path, save_path)) + '/'
    dates = np.datetime_as_string(xr_data.time.values,unit='D')
    for date in np.unique(dates):
        date = str(date) # cast numpy_str to str
        xrd = xr_data.sel(time=slice(date,date))
        filename = save_path + filename_prefix + '_' + date.replace('-','_') + '.nc'
        if os.path.exists(filename):
            if overwrite:
                print_verbose('Writing data for date '+ date + 'to file '+filename)
                xrd.to_netcdf(path=filename)
            else:
                pass
                print_verbose('Data for date ' + date + ' already exists in file ' + filename)
        else:
            print_verbose('Writing data for date ' + date + ' to file ' + filename)
            xrd.to_netcdf(path=filename)
            pass
    return

In [4]:
retrieve_data()

Data for date 2012-01-01 already exists in file /export/home1/jbenjami/Research_Thompson/Data/HF_Radar/2km/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best_2012_01_01.nc
Data for date 2012-01-02 already exists in file /export/home1/jbenjami/Research_Thompson/Data/HF_Radar/2km/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best_2012_01_02.nc
Data for date 2012-01-03 already exists in file /export/home1/jbenjami/Research_Thompson/Data/HF_Radar/2km/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best_2012_01_03.nc
Data for date 2012-01-04 already exists in file /export/home1/jbenjami/Research_Thompson/Data/HF_Radar/2km/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best_2012_01_04.nc
Data for date 2012-01-05 already exists in file /export/home1/jbenjami/Research_Thompson/Data/HF_Radar/2km/HFRADAR_US_West_Coast_2km_Resolution_Hourly_RTV_best_2012_01_05.nc
Data for date 2012-01-06 already exists in file /export/home1/jbenjami/Research_Thompson/Data/HF_Radar/2km/HFRADAR_US_West_Coast_2